In [ ]:
import numpy as np
import numpy.linalg as la
import cv2
import matplotlib.pyplot as plt


In [ ]:
n_views = 101
n_features = 215
data = np.loadtxt('data/data_matrix.txt')

### 1. Create the centroid subtracted matrix D from feature coordinates
- Given m images and n features
- $m$ = n_views = 101 and $n$ = n_features = 215
- D is measurement matrix of size $2m \times n$ which is $202\times 215$

In [ ]:
centroids = np.mean(data, axis=1).reshape((-1, 1, 2))
x = np.swapaxes(data.reshape((n_views, 2, n_features)), 1, 2)
x_hat = x - centroids
D = np.swapaxes(x_hat, 1, 2).reshape(n_views * 2, n_features)

### 2. Compute SVD of D and enforce rank constraint 
 - $D = UWV^{T}$
 - Create $U_{3}$ and $V_{3}$ by taking the first columns of $U$ and $V$
 - Create $W_{3}$ by taking the upper left $3\times3$ block of $W$
 - Create motion ($M$) and shape ($S$) matrices:
 $$M = U_{3}W_{3}^{\frac{1}{2}} \qquad S = W_{3}^{\frac{1}{2}}V_{3}^{T}$$  

In [ ]:
U, W, V = la.svd(D)

U = U[:, 0:3]
V = V[0:3, :]
W = np.diag(np.squeeze(W)[0:3])

W_sqrt = np.sqrt(W)

M = U @ W_sqrt

S = W_sqrt @ V


In [ ]:
M.shape

### 3. Eliminate affine ambiguity by enforcing orthonormality constraint 
If the motion matrix for an image is $M=\begin{bmatrix} m_{1}^{T} \\ m_{2}^{T} \end{bmatrix}$ then we solve for $L=CC^{T}$ which is symmetric and has 6 variables.

$$
m_{1}L\; m_{2}^{T} = 
\begin{bmatrix}
m_{11} & m_{12} & m_{13} 
\end{bmatrix}
\begin{bmatrix}
l_{11} & l_{12} & l_{13} \\
l_{12} & l_{22} & l_{23} \\
l_{13} & l_{23} & l_{33} \\
\end{bmatrix}
\begin{bmatrix}
m_{11} \\
m_{12} \\
m_{13} 
\end{bmatrix} = 1
$$
can be rewritten as :
$$
\begin{bmatrix}
m_{11}^{2} & 2m_{11}m_{12} & 2m_{11}m_{13} & m_{12}^{2} & 2m_{12}m_{13} & m_{13}^{2}
\end{bmatrix}
\begin{bmatrix}
l_{11} \\
l_{12} \\
l_{13} \\
l_{22} \\
l_{23} \\
l_{33}
\end{bmatrix} = 1
$$
The same can be done for: 
$$
m_{2}L\; m_{2}^{T} = 1 \\
m_{1}L\; m_{2}^{T} = 0 \\
$$

- Finnaly $C$ can be recovered from $L$ by SVD or Cholesky
decomposition

In [ ]:
A = np.zeros((3 * n_views, 6))
B = np.zeros((3 * n_views, 1))

for i in range(n_views):
    line_1 = i * 3
    line_2 = i * 3 + 1
    line_3 = i * 3 + 2

    mi1 = M[i * 2]
    mi2 = M[i * 2 + 1]

    m11, m12, m13 = mi1
    m21, m22, m23 = mi2

    A[line_1] = np.array([m11 ** 2,
                          2 * m11 * m12,
                          2 * m11 * m13,
                          m12 ** 2,
                          2 * m12 * m13,
                          m13 ** 2])
    A[line_2] = np.array([m21 ** 2,
                          2 * m21 * m22,
                          2 * m21 * m23,
                          m22 ** 2,
                          2 * m22 * m23,
                          m23 ** 2])
    A[line_3] = np.array([m11 * m21,
                          m12 * m21 + m11 * m22,
                          m13 * m21 + m11 * m23,
                          m12 * m22,
                          m13 * m22 + m12 * m23,
                          m13 * m23])

    B[line_1] = 1
    B[line_2] = 1
    B[line_3] = 0

X = la.lstsq(A, B, rcond=-1)[0]

l11, l12, l13, l22, l23, l33 = np.squeeze(X)

L = np.array([
    [l11, l12, l13],
    [l12, l22, l23],
    [l13, l23, l33]
])

C = la.cholesky(L)


### 4. Update M and S

- $M^{\prime}=MC$ and $S^{\prime}=C^{-1}S$  

In [ ]:
M_prime = M @ C
S_prime = la.inv(C) @ S